# 1. 설계
1. 데이터 준비
    - 개념 설명을 위한 하드코딩된 예시 샘플
    - 간단한 문장 및 그에 대한 품사 태그 정의
2. 어휘 사전(Vocabulary) 및 태그 사전(Tab Vocabulary) 구성
    - 토큰 -> 숫자 변환을 위한 word2index, index2word 매핑 생성
    - 품사 태그 - 숫자 매핑(e.g, tag2index, index2tag)
3. Pytorch Dataset & DataLoader 정의
    - Dataset 클래스 활용, 샘플(토큰 시퀀스) 및 정답(품사 태그 시퀀스) 반환
    - DataLoader 활용, 배치(batch) 단위 데이터 처리
4. 모델(Encoder-Only RNN) 설계
    - Embedding Layer: 입력 단어 임베딩 벡터 변환
    - RNN: 시퀀스 정보 인코딩
    - Fully-Connected Layer: RNN의 은닉 상태(hidden state) 출력을 받아 각 시점(time-step)마다 품사 태그 예측
    - 특징: 일반적인 seq2seq 모델(Encoder-Decoder)와 달리, Encoding-only 접근에서는 인코더에서 바로 분류 수행
5. 학습(Training) 루프
    - 손실 함수(loss function): 토큰 단위 교차 엔토로피 손실
    - 옵티마이저(optimizer): Adam, SGD 등
    - Epoch마다 전체 데이터 반복, 예측-손실 계산-역전파-가중치 갱신 수행
6. 추론(Inference) 및 성능 확인
    - 학습된 모델의 샘플 데이터 예측 결과 확인

# 2. 코드 구현 예시

In [ ]:
# 필요 라이브러리 호출
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

: 

In [ ]:
# 1. 예시 데이터 준비
sentences = [
    ["나는", "밥을", "먹는다"],
    ["너는", "공부를", "한다"],
    ["그녀는", "책을", "읽는다"],
],

pos_tags = [
    ["PRON", "NOUN", "VERB"],
    ["PRON", "NOUN", "VERB"],
    ["PRON", "NOUN", "VERB"],
]

In [ ]:
# 2. Vocabulary 생성
word_list = set([word for sent in sentences for word in sent])
word_list = list(word_list)
word2index = {w: i+1 for i, w in enumerate(word_list)}
index2word = {i: w for w, i in word2index.items()}

tag_list = set([tag for tags in pos_tags for tag in tags])
tag_list = list(tag_list)
tag2index = {t: i for i, t in enumerate(tag_list)}
index2tag = {i: t for t, i in tag2index.items()}

vocab_size = len(word2index) + 1
tag_size = len(tag2index)

print("Word Vocabulary:", word2index)
print("Tag Vocabulary:", tag2index)

In [ ]:
# 3. Dataset 및 DataLoader 생성
def encode_sentence(sentence):
    """
    단어 사전 활용, 문장 -> 숫자 리스트 변환
    예: ["나는", "밥을", "먹는다"] -> [2, 5, 1]
    """
    return [word2index[w] for w in sentence]

def encode_tag(tags):
    """
    태그 사전 활용, 태그 -> 숫자 리스트 변환
    예: ["PRON", "NOUN", "VERB"] -> [1, 0, 2]
    """
    return [tag2index[t] for t in tags]

class PosDataset(Dataset):
    def __init__(self, sentences, pos_tags):
        super(PosDataset, self).__init__()
        self.sentences = sentences
        self.pos_tags = pos_tags

        # 미리 숫자 인코딩
        self.encoded_sentences = [encode_sentence(sent) for sent in sentences]
        self.encoded_tags = [encode_tag(tags) for tags in pos_tags]

    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        # 각 샘플을 (입력 시퀀스, 타깃 시퀀스) 형태 변환
        return (
            torch.tensor(self.encoded_sentences[idx], dtype=torch.long),
            torch.tensor(self.encoded_tags[idx], dtype=torch.long)
        )